#### MIRA Course -- Non-rigid registration 
Todo's:
1.	Try to localize the points in the given images.
2.	Calculate the initial error without registration
3.	Calculate non rigid registration on intensity images and on the points
4.	Calculate the non-rigid registration errors. 

In [39]:
# Import Libraries
import os
from pathlib import Path
import numpy as np
import pandas as pd
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle

In [40]:
# Read nifti files
def read_nifti_file(filepath):
    # Read the image using SimpleITK
    itkimage = sitk.ReadImage(filepath)
    # Convert the image to a  numpy array first and then shuffle the dimensions to get axis in the order z,y,x
    ct_scan = sitk.GetArrayFromImage(itkimage)
    # Read the origin of the ct_scan, will be used to convert the coordinates from world to voxel and vice versa.
    origin = np.array(list(reversed(itkimage.GetOrigin())))
    # Read the spacing along each dimension
    spacing = np.array(list(reversed(itkimage.GetSpacing())))
    return ct_scan, origin, spacing

In [41]:
image = sitk.ReadImage("data/copd1_iBHCT.nii.gz")
print(image.GetSize())
print(image.GetOrigin())
print(image.GetSpacing())
print(image.GetDirection())
print(image.GetNumberOfComponentsPerPixel())

(512, 512, 121)
(0.0, 0.0, 0.0)
(0.6200000047683716, 0.6200000047683716, 2.5)
(1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, -1.0)
1


In [56]:
# Read landmarks 
def get_landmark(p, i):
    """get the landmarks of the patient

    Args:
        p (str): inhale or exhale (i or e)
        i (int): patient number

    Returns:
        _type_: pandas dataframe
    """
    file= f"data/copd{i}/copd{i}_300_{p}BH_xyz_r1.txt"
    if p == "i":
        landmark = pd.read_csv(file, sep="\t", header=None)
    else:
        landmark = pd.read_csv(file, sep="\t ", header=None)
    return np.array(landmark)

get_landmark(p="i", i=1)


array([[188, 260,   4],
       [179, 281,   7],
       [187, 253,   8],
       [161, 315,   9],
       [173, 241,   9],
       [317, 242,  10],
       [310, 319,  10],
       [282, 273,  11],
       [198, 234,  11],
       [215, 256,  12],
       [190, 311,  12],
       [303, 319,  12],
       [283, 231,  13],
       [181, 235,  13],
       [156, 234,  13],
       [188, 295,  13],
       [359, 278,  14],
       [342, 345,  15],
       [369, 285,  15],
       [185, 228,  15],
       [195, 334,  15],
       [157, 315,  15],
       [217, 278,  16],
       [379, 280,  16],
       [330, 233,  17],
       [371, 305,  17],
       [162, 348,  17],
       [309, 305,  18],
       [383, 265,  18],
       [367, 261,  19],
       [351, 292,  19],
       [290, 227,  19],
       [147, 287,  19],
       [235, 263,  19],
       [147, 351,  19],
       [130, 339,  20],
       [180, 263,  20],
       [227, 265,  21],
       [189, 228,  21],
       [120, 278,  22],
       [135, 285,  22],
       [392, 256

In [51]:
# Calculate TRE
def calculate_tre(fixed_points, moving_points):
    tre = np.sqrt(np.sum((fixed_points - moving_points) ** 2, axis=1))
    return tre

for k in range(1,5):
    fixed_points = get_landmark(p="i", i=k)
    moving_points = get_landmark(p="e", i=k)
    mean = np.mean(calculate_tre(fixed_points, moving_points))
    std = np.std(calculate_tre(fixed_points, moving_points))
    print(f"Mean: {mean}, Std: {std}")

Mean: 39.76142002901733, Std: 17.40860354181402
Mean: 31.286046004532576, Std: 10.20996206664213
Mean: 11.50253275543796, Std: 5.573006986963454
Mean: 33.12005638151803, Std: 16.17380181914774


C:\Users\Lenovo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)
